In [42]:
import os
import re
import glob
import time
from datetime import datetime
from typing import Dict, List
from dotenv import load_dotenv

import pandas as pd
import numpy as np
import requests
import yfinance as yf
from bs4 import BeautifulSoup

import google.generativeai as genai

from Modelyaga import big_model_baba

In [43]:
load_dotenv()
KEY = os.environ['GEMINI']

In [44]:
# Configure Gemini API
genai.configure(api_key=KEY)  # Replace with your actual API key

# Set up the model
generation_config = {
    "temperature": 0.7,
    "top_p": 1,
    "top_k": 32,
    "max_output_tokens": 4096,
}

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
]

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash",
    generation_config=generation_config,
    safety_settings=safety_settings
)

In [45]:
def cleanup():
    # Define the directory
    output_dir = "TICKER_DATA"

    # Find all CSV files in the directory
    files = glob.glob(os.path.join(output_dir, "*.csv"))

    # Delete each file
    for file in files:
        try:
            os.remove(file)
            print(f"Deleted: {file}")
        except Exception as e:
            print(f"Error deleting {file}: {e}")

    print("Cleanup completed. All CSV files deleted.")

def cleanup_csv_data(file):
    # Read the CSV file
    df = pd.read_csv(file)

    # List of columns to set to NaN (all except Symbol, Company Name, Sector)
    cols_to_nan = ['Previous Price', 'Predicted Price', 'Change ($)', 'Change (%)', 
                'New/Old Ratio', 'MSE', 'R2', 'Sentiment Score', 'Sentiment Validity']

    # Set these columns to NaN
    df[cols_to_nan] = np.nan

    # Save or display the result
    df.to_csv(file, index=False)

In [46]:
def get_ticker_data():
    # Create a directory to save data
    output_dir = "TICKER_DATA"
    os.makedirs(output_dir, exist_ok=True)

    # Load tickers from CSV (Assuming 'tickers.csv' has a column 'Symbol')
    tickers_df = pd.read_csv("tickers.csv")
    tickers = tickers_df["Symbol"].tolist()

    # Define start and end dates (past 1 year)
    end_date = pd.to_datetime("today").strftime('%Y-%m-%d')
    start_date = (pd.to_datetime("today") - pd.DateOffset(years=1)).strftime('%Y-%m-%d')

    # Fetch historical stock prices for each ticker and save separately
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            history = stock.history(start=start_date, end=end_date, interval="1d")
            if not history.empty:
                history.reset_index(inplace=True)
                history["Symbol"] = ticker  # Add ticker column
                
                # Save as CSV
                file_path = os.path.join(output_dir, f"{ticker}.csv")
                history.to_csv(file_path, index=False)
                print(f"Saved: {file_path}")
            else:
                print(f"No data for {ticker}")

        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    print(f"All stock data saved in '{output_dir}' directory.")

In [47]:
def predict_and_update():
    # Read the tickers information
    tickers_info = pd.read_csv('tickers_ml.csv')

    # Initialize lists to store all data
    results = []

    ticker_dir = "./TICKER_DATA/"
    ticker_files = os.listdir("./TICKER_DATA/")

    # Process each file
    for ticker_file in ticker_files:
        ticker = ticker_file.split('.')[0]
        file_path = os.path.join(ticker_dir, ticker_file)
        
        # Get predictions and stats
        pred, stat = big_model_baba(file_path)
        
        # Read last actual price from CSV
        df = pd.read_csv(file_path)
        prev_price = df['Close'].iloc[-1]
        
        # Calculate metrics
        change_dollar = pred - prev_price
        change_percent = (change_dollar / prev_price) * 100
        new_old_ratio = pred / prev_price
        
        # Get company info from tickers_info
        company_info = tickers_info[tickers_info['Symbol'] == ticker].iloc[0]
        
        # Create result dictionary
        result = {
            'Symbol': ticker,
            'Company Name': company_info['Company Name'],
            'Sector': company_info['Sector'],
            'Previous Price': prev_price,
            'Predicted Price': pred,
            'Change ($)': change_dollar,
            'Change (%)': change_percent,
            'New/Old Ratio': new_old_ratio,
            'MSE': stat['mse'],
            'R2': stat['r2']
            # Add any other stats from stat dictionary here
        }
        
        results.append(result)
        
        # Print results in one row (optional)
        # print(f"{ticker:<6} | {prev_price:10.2f} | {pred:10.2f} | {change_dollar:8.2f} | {change_percent:8.2f}% | {new_old_ratio:8.4f} | {stat['mse']:8.2f} | {stat['r2']:8.4f}")

        # Create DataFrame from results
        results_df = pd.DataFrame(results)

        # Reorder columns if needed
        columns_order = ['Symbol', 'Company Name', 'Sector', 'Previous Price', 'Predicted Price', 
                        'Change ($)', 'Change (%)', 'New/Old Ratio', 'MSE', 'R2']
        results_df = results_df[columns_order]

        # Save to CSV if desired
        results_df.to_csv('tickers_ml.csv', index=False)

In [48]:
def get_news(ticker):
    ticker = yf.Ticker(ticker)

    news = ticker.news

    news_text = ""

    for item in news:
        news_text += f"Title: {item["content"]["title"]}\n" + f"Summary: {item["content"]["summary"]}\n"

    return news_text

In [49]:
def analyze_news_sentiment(news_text: str) -> List[Dict]:
    """
    Analyzes financial news text and returns sentiment analysis for mentioned tickers
    
    Args:
        news_text: The financial news text to analyze
        
    Returns:
        List of dictionaries with ticker, sentiment score, and sentiment validity
        Format: [{"ticker": str, "sentiment score": float, "sentiment validity": float}]
    """
    prompt = f"""Analyze this financial news text and perform the following tasks:
    
    1. Identify all stock tickers mentioned (e.g., AAPL, MSFT, GOOGL)
    2. For each ticker, determine the sentiment (positive/negative/neutral)
    3. Assign a sentiment score between 0 (most negative) and 1 (most positive)
    4. Assign a sentiment validity score between 0 (low confidence) and 1 (high confidence)
       based on how clearly the sentiment is expressed for that ticker
    
    Return ONLY a JSON-formatted list of dictionaries with these keys:
    - "ticker" (the stock symbol)
    - "sentiment score" (0-1)
    - "sentiment validity" (0-1)
    
    News text to analyze:
    {news_text}
    """
    
    try:
        response = model.generate_content(prompt)
        
        # Extract JSON from the response
        json_str = response.text.strip().replace('```json', '').replace('```', '').strip()
        
        # Parse the JSON string into Python objects
        import json
        result = json.loads(json_str)
        
        # Validate the structure
        if not isinstance(result, list):
            raise ValueError("Response is not a list")
            
        for item in result:
            if not all(key in item for key in ["ticker", "sentiment score", "sentiment validity"]):
                raise ValueError("Missing required keys in response")
                
        return result
        
    except Exception as e:
        print(f"Error analyzing news: {e}")
        return []

In [50]:
def update_tickers_with_sentiment(csv_path: str, sentiment_results: List[Dict]) -> pd.DataFrame:
    """
    Updates the tickers CSV with sentiment analysis results
    
    Args:
        csv_path: Path to the tickers CSV file
        sentiment_results: List of sentiment analysis dictionaries from Gemini
        
    Returns:
        Updated pandas DataFrame
    """
    try:
        # Read the CSV file
        df = pd.read_csv(csv_path)
        
        # Initialize new columns if they don't exist
        if 'Sentiment Score' not in df.columns:
            df['Sentiment Score'] = None
        if 'Sentiment Validity' not in df.columns:
            df['Sentiment Validity'] = None
        
        # Create a mapping from sentiment results for quick lookup
        sentiment_map = {
            item['ticker']: {
                'score': item['sentiment score'],
                'validity': item['sentiment validity']
            }
            for item in sentiment_results
        }
        
        # Update the DataFrame
        for index, row in df.iterrows():
            ticker = row['Symbol']
            if ticker in sentiment_map:
                df.at[index, 'Sentiment Score'] = sentiment_map[ticker]['score']
                df.at[index, 'Sentiment Validity'] = sentiment_map[ticker]['validity']
        
        # Optionally save back to CSV
        df.to_csv(f"tickers_news.csv", index=False)
        
        return df
        
    except Exception as e:
        print(f"Error updating tickers CSV: {e}")
        return pd.DataFrame()

In [51]:
def process_tickers_news():
    for ticker in os.listdir("./TICKER_DATA/"):
        news_text = get_news(ticker[:-4])
        sentiment_results = analyze_news_sentiment(news_text)
        update_tickers_with_sentiment("./tickers_news.csv", sentiment_results)
        time.sleep(2)

In [52]:
cleanup()
cleanup_csv_data("./tickers.csv")
cleanup_csv_data("./tickers_ml.csv")
cleanup_csv_data("./tickers_news.csv")

Cleanup completed. All CSV files deleted.


In [53]:
get_ticker_data()

Saved: TICKER_DATA/AAPL.csv
Saved: TICKER_DATA/V.csv
Saved: TICKER_DATA/JNJ.csv
Saved: TICKER_DATA/PYPL.csv
Saved: TICKER_DATA/JPM.csv
Saved: TICKER_DATA/WMT.csv
Saved: TICKER_DATA/TSLA.csv
Saved: TICKER_DATA/AMD.csv
Saved: TICKER_DATA/META.csv
Saved: TICKER_DATA/MSFT.csv
Saved: TICKER_DATA/T.csv
Saved: TICKER_DATA/AMZN.csv
Saved: TICKER_DATA/VZ.csv
Saved: TICKER_DATA/ADBE.csv
Saved: TICKER_DATA/KO.csv
Saved: TICKER_DATA/CRM.csv
Saved: TICKER_DATA/PG.csv
Saved: TICKER_DATA/PFE.csv
Saved: TICKER_DATA/GOOGL.csv
Saved: TICKER_DATA/NVDA.csv
All stock data saved in 'TICKER_DATA' directory.


### Parallely Compute

In [54]:
import multiprocessing

def main():
    # Create two separate processes
    process1 = multiprocessing.Process(target=predict_and_update)
    process2 = multiprocessing.Process(target=process_tickers_news)
    
    # Start both processes
    process1.start()
    process2.start()
    
    # Wait for both processes to complete
    process1.join()
    process2.join()

main()

### Merge ML Preds and Sentiment Analysis

In [69]:
def merge_stock_data(prediction_file, sentiment_file, output_file):
    # Read the files
    predictions = pd.read_csv(prediction_file)
    sentiments = pd.read_csv(sentiment_file)
    
    # Identify columns to add from sentiment file (excluding overlapping columns)
    sentiment_cols_to_add = [col for col in sentiments.columns 
                           if col not in predictions.columns]
    
    # Merge while keeping all prediction columns and only adding sentiment columns
    merged_data = predictions.merge(
        sentiments[['Symbol', 'Company Name', 'Sector'] + sentiment_cols_to_add],
        on=['Symbol', 'Company Name', 'Sector'],
        how='left'
    )
    
    # Save the merged data
    merged_data.to_csv(output_file, index=False)
    print(f"Merged data saved to {output_file}")
    
    return merged_data

In [80]:
prediction_csv = "tickers_ml.csv"
sentiment_csv = "tickers_news.csv"
output_csv = "tickers.csv"

result = merge_stock_data(prediction_csv, sentiment_csv, output_csv)

Merged data saved to tickers.csv


### Relative MSE

In [84]:
input_csv = "tickers.csv"
df = pd.read_csv(input_csv)
df['Relative_MSE'] = (df['MSE'] / (df['Previous Price'] ** 2)) * 100
df.to_csv(input_csv, index=False)

In [85]:
ddff = pd.read_csv("./tickers.csv")
ddff

,Symbol,Company Name,Sector,Previous Price,Predicted Price,Change ($),Change (%),New/Old Ratio,MSE,R2,Sentiment Score,Sentiment Validity,Relative_MSE
0,AAPL,Apple Inc.,Technology,223.850006,226.812060,2.962051,1.323230,1.013232,23.410751,0.848195,0.70,0.6,0.046720
1,V,Visa Inc.,Financials,349.859985,345.341520,-4.518463,-1.291506,0.987085,78.145271,0.054124,0.75,0.9,0.063843
2,JNJ,Johnson & Johnson,Healthcare,163.130005,162.204250,-0.925751,-0.567493,0.994325,12.136504,-1.574536,0.80,0.9,0.045606
3,PYPL,PayPal Holdings,Fintech,68.860001,70.977630,2.117630,3.075269,1.030753,5.038738,0.393294,0.20,0.6,0.106264
4,JPM,JPMorgan Chase & Co.,Financials,248.119995,249.891140,1.771149,0.713827,1.007138,164.340439,0.102823,0.70,0.9,0.266944
5,WMT,Walmart Inc.,Consumer Staples,85.629997,86.901450,1.271454,1.484823,1.014848,34.160824,-0.107012,0.60,0.7,0.465883
6,TSLA,Tesla Inc.,Consumer Discretionary,273.130005,272.615000,-0.515015,-0.188560,0.998114,1694.519043,-0.220669,0.40,0.7,2.271473
7,AMD,Advanced Micro Devices,Semiconductors,106.650002,106.481020,-0.168983,-0.158447,0.998416,184.547256,-4.914134,0.20,0.8,1.622504
8,META,Meta Platforms (Facebook),Communication Services,602.580017,622.114260,19.534241,3.241767,1.032418,315.067383,0.742488,0.60,0.6,0.086771
9,MSFT,Microsoft Corp.,Technology,390.579987,394.381470,3.801483,0.973292,1.009733,64.211922,0.266821,0.70,0.8,0.042092


### Signal Generation

In [93]:
import pandas as pd

# Define weights and normalization constant
alpha = 1.0  # weight for price momentum
beta = 5.0   # weight for sentiment term
gamma = 10.0  # weight for new/old ratio term
delta = 1.0  # weight for R2
epsilon = 1.0  # weight for MSE term

# Read CSV file
df = pd.read_csv('tickers.csv')

# Compute the price momentum: predicted percentage change in price
df['Price_Momentum'] = (df['Predicted Price'] - df['Previous Price']) / df['Previous Price']

# Compute the sentiment term: product of sentiment score and its validity
df['Sentiment_Term'] = df['Sentiment Score'] * df['Sentiment Validity']

# Compute the adjusted new/old ratio term: centered around 0
df['Relative_Ratio'] = df['New/Old Ratio'] - 1

# Compute the composite signal using the proposed formula
df['Signal'] = (
    alpha * df['Price_Momentum'] +
    beta * df['Sentiment_Term'] +
    gamma * df['Relative_Ratio'] +
    delta * df['R2'] -
    epsilon * (df['Relative_MSE'])
)

df = df.sort_values(by='Signal', ascending=False)

# Display the updated DataFrame
df.to_csv("final.csv", index=False)

In [94]:
df

,Symbol,Company Name,Sector,Previous Price,Predicted Price,Change ($),Change (%),New/Old Ratio,MSE,R2,Sentiment Score,Sentiment Validity,Relative_MSE,Price_Momentum,Sentiment_Term,Relative_Ratio,Signal
19,NVDA,NVIDIA Corp.,Semiconductors,111.430000,115.248790,3.818787,3.427072,1.034271,21.752396,0.701748,0.70,0.9,0.175187,0.034271,0.630,0.034271,4.053539
16,PG,Procter & Gamble,Consumer Staples,168.710007,167.408550,-1.301453,-0.771414,0.992286,3.971182,0.676363,0.75,0.9,0.013952,-0.007714,0.675,-0.007714,3.952556
11,AMZN,Amazon.com Inc.,Consumer Discretionary,201.360001,205.537600,4.177597,2.074691,1.020747,36.368404,0.546737,0.70,0.9,0.089697,0.020747,0.630,0.020747,3.835256
15,CRM,Salesforce Inc.,Cloud Computing,277.809998,284.219240,6.409241,2.307059,1.023071,88.025177,0.540207,0.70,0.9,0.114054,0.023071,0.630,0.023071,3.829929
1,V,Visa Inc.,Financials,349.859985,345.341520,-4.518463,-1.291506,0.987085,78.145271,0.054124,0.75,0.9,0.063843,-0.012915,0.675,-0.012915,3.223215
13,ADBE,Adobe Inc.,Software,396.149994,404.133200,7.983215,2.015200,1.020152,463.948761,0.310164,0.65,0.9,0.295632,0.020152,0.585,0.020152,3.161204
9,MSFT,Microsoft Corp.,Technology,390.579987,394.381470,3.801483,0.973292,1.009733,64.211922,0.266821,0.70,0.8,0.042092,0.009733,0.560,0.009733,3.131792
4,JPM,JPMorgan Chase & Co.,Financials,248.119995,249.891140,1.771149,0.713827,1.007138,164.340439,0.102823,0.70,0.9,0.266944,0.007138,0.630,0.007138,3.064399
0,AAPL,Apple Inc.,Technology,223.850006,226.812060,2.962051,1.323230,1.013232,23.410751,0.848195,0.70,0.6,0.046720,0.013232,0.420,0.013232,3.047030
8,META,Meta Platforms (Facebook),Communication Services,602.580017,622.114260,19.534241,3.241767,1.032418,315.067383,0.742488,0.60,0.6,0.086771,0.032418,0.360,0.032418,2.812312
